# Advent of Code 2021

Never again.

That's what I said in 2019. I both love https://adventofcode.com and hate that the puzzles are so beautifully put together that I can't resist doing one more little one - even though I really don't need anything else going on in my life over the festive season. Perhaps I'll just take a look at day 1... And then...

Well if I'm going to do this I might as well record feelings I've had along the way and any insights into how I could approach other problems better.

I'm no way committing to completing it this year. (Come on - you don't have to do this)

Merry Christmas, J.

In [2]:
from collections import (
    Counter,
    defaultdict,
    deque,
    namedtuple
)

from dataclasses import dataclass
import re

In [3]:
def get_puzzle_input(file, strip_lines=True, remove_blank_lines=True):
    """Return the puzzle input as a list."""
    with open(file) as fh:
        lines = fh.readlines()
    
    if strip_lines:
        lines = [l.strip() for l in lines]
    
    if remove_blank_lines:
        lines = [l for l in lines if l]
        
        
    return lines

## Day 1 Sonar Sweep

https://adventofcode.com/2021/day/1

Count the number of times a depth measurement increases from the previous measurement. (There is no measurement before the first measurement.)

In [4]:
test_input = [
    199,
    200,
    208,
    210,
    200,
    207,
    240,
    269,
    260,
    263
]
real_input = get_puzzle_input('input.txt')
real_input = [int(i) for i in real_input]

In [5]:
def num_increases(puzzle_input):
    num_increases = 0
    current_num = None
    for val in puzzle_input:
        if current_num is not None and val > current_num:
            num_increases += 1
        current_num = val

    return num_increases

In [6]:
num_increases(test_input), num_increases(real_input)

(7, 1754)

### Day 1 Part 2
Instead, consider sums of a three-measurement sliding window

In [7]:
def windows(puzzle_input, window_size):
    output = []
    puzzle_len = len(puzzle_input)
    for puzzle_idx in range(puzzle_len):
        window = []
        for window_idx in range(window_size):
            idx = puzzle_idx + window_idx
            if idx < puzzle_len:
                window.append(puzzle_input[idx])
        output.append(tuple(window))
    return output

In [8]:
windows(test_input, 3)

[(199, 200, 208),
 (200, 208, 210),
 (208, 210, 200),
 (210, 200, 207),
 (200, 207, 240),
 (207, 240, 269),
 (240, 269, 260),
 (269, 260, 263),
 (260, 263),
 (263,)]

In [9]:
windowed_test_input = [sum(x) for x in windows(test_input, 3)]
num_increases(windowed_test_input)

5

In [10]:
windowed_real_input = [sum(x) for x in windows(real_input, 3)]
num_increases(windowed_real_input)

1789

### Day 1 Observations

I skipped last year's AOC and I realise that I always go into each of these puzzles with the expectation that I won't have a clue where to start and everyone else will be able to see the solution. Also interesting that I started on the windowing solution just as my train was pulling into the station with a 'write it and see what happens' mind set; it was turning into a real mess. During the next 10 minute bus ride I had a much clearer idea of what I was going to do. Sometimes it pays to walk away from the keyboard.

## Day 2 Dive!

https://adventofcode.com/2021/day/2

Calculate the horizontal position and depth you would have after following the planned course.

In [11]:
@dataclass(frozen=True)
class Position:
    horizontal: int = 0
    depth: int = 0
    
    def move(self, command):
        movement_map = {
            'forward': (1, 0),
            'down': (0, 1),
            'up': (0, -1),
        }
        input_re = re.compile(r'(\S+) ([0-9-]+)')
        match = input_re.match(command)
        if not match:
            print(command)
        
        direction = match.groups()[0]
        amount = int(match.groups()[1])
        
        diff = movement_map[direction]
        return Position(
            self.horizontal + diff[0] * amount,
            self.depth + diff[1] * amount,
        )


In [14]:
assert Position(0, 0).move('down 12') == Position(0, 12)
assert Position(0, 12).move('up 12') == Position(0, 0)
assert Position(100, 12).move('forward 12') == Position(112, 12)

In [17]:
test_directions = [
    'forward 5',
    'down 5',
    'forward 8',
    'up 3',
    'down 8',
    'forward 2'
]
real_directions = get_puzzle_input('input_2.txt')

In [20]:
def play(directions, initial_position):
    position = initial_position
    for dir in directions:
        position = position.move(dir)
    return position.horizontal * position.depth


In [22]:
play(test_directions, Position(0, 0)), play(real_directions, Position(0, 0))

(150, 1524750)

### Day 2 Part 2

Calculate using the new interpretation as up and down as tilt.

In [24]:
@dataclass(frozen=True)
class Position2:
    horizontal: int = 0
    depth: int = 0
    aim: int = 0
    
    def move(self, command):
        movement_map = {
            'forward': (1, self.aim, 0),
            'down': (0, 0, 1),
            'up': (0, 0, -1),
        }
        input_re = re.compile(r'(\S+) ([0-9-]+)')
        match = input_re.match(command)
        if not match:
            print(command)
        
        direction = match.groups()[0]
        amount = int(match.groups()[1])
        
        diff = movement_map[direction]
        return Position2(
            self.horizontal + diff[0] * amount,
            self.depth + diff[1] * amount,
            self.aim + diff[2] * amount,
        )

In [25]:
play(test_directions, Position2(0, 0)), play(real_directions, Position2(0, 0))

(900, 1592426537)

### Day 2 Observations

Frozen dataclasses have been the killer missing feature in Python. Once you've worked with Scala case classes or Kotlin data classes, you can't imagine why a language wouldn't have such a feature (I guess I wasn't aware of them when doing 2018 AOC!)

## Day 3 Binary Diagnostic

https://adventofcode.com/2021/day/3

Use the binary numbers in your diagnostic report to calculate the gamma rate and epsilon rate, then multiply them together. What is the power consumption of the submarine? (Be sure to represent your answer in decimal, not binary.)

In [26]:
test_input = [
    '00100',
    '11110',
    '10110',
    '10111',
    '10101',
    '01111',
    '00111',
    '11100',
    '10000',
    '11001',
    '00010',
    '01010',
]
real_input = get_puzzle_input('input3.txt')

In [27]:
def gamma_rate(input):
    num_numbers = len(input)
    num_figures = len(input[0])
    running_count = [0] * num_figures
    binaries = [int(x, base=2) for x in input]
    
    for binary in binaries:
        for figure in range(num_figures):
            mask = 2**figure
            is_set = (binary & mask) // mask
            running_count[figure] += is_set

    return sum(
        val * 2 // num_numbers * 2**idx
        for (idx, val)
        in enumerate(running_count)
    )

In [28]:
def epsilon_rate(gamma_rate, num_figures):
    return (2**num_figures - 1) ^ gamma_rate

In [31]:
test_gamma_rate = gamma_rate(test_input)
test_epsilon_rate = epsilon_rate(test_gamma_rate, len(test_input[0]))
test_gamma_rate, test_epsilon_rate, test_gamma_rate * test_epsilon_rate


(22, 9, 198)

In [32]:
real_gamma_rate = gamma_rate(real_input)
real_epsilon_rate = epsilon_rate(real_gamma_rate, len(real_input[0]))
real_gamma_rate, real_epsilon_rate, real_gamma_rate * real_epsilon_rate

(3808, 287, 1092896)

### Day 3, part 2

Use the binary numbers in your diagnostic report to calculate the oxygen generator rating and CO2 scrubber rating, then multiply them together. What is the life support rating of the submarine? (Be sure to represent your answer in decimal, not binary.)

In [33]:
def figure_filter(figures, figure_idx, is_most_common): 
    running_count = 0
    num_numbers = len(figures)
    mask = 2**figure_idx
    ones = []
    zeroes = []

    for figure in figures:
        is_set = (figure & mask) // mask
        running_count += is_set
        if is_set:
            ones.append(figure)
        else:
            zeroes.append(figure)

    more_ones = running_count * 2 >= num_numbers
    
    if (
        more_ones and is_most_common or
        not more_ones and not is_most_common
    ):
        return ones
    else:
        return zeroes

In [34]:
def part2(figures, is_most_common):
    int_figures = [int(x, base=2) for x in figures]
    num_figures = len(figures[0])
    # Count down to look at the most sig bit first
    for figure_idx in range(num_figures - 1, -1, -1):
        int_figures = figure_filter(int_figures, figure_idx, is_most_common)
        
        if len(int_figures) <= 1:
            break
    
    return int_figures 

In [36]:
oxygen_rating = part2(test_input, True)
co_rating = part2(test_input, False)
oxygen_rating, co_rating, oxygen_rating[0] * co_rating[0]

([23], [10], 230)

In [38]:
oxygen_rating = part2(real_input, True)
co_rating = part2(real_input, False)
oxygen_rating, co_rating, oxygen_rating[0] * co_rating[0]

([3443], [1357], 4672151)

### Day 3 Observations

Think I made the right call in working with ints rather than strings in part 1.


## Day 4 Giant Squid

https://adventofcode.com/2021/day/4

To guarantee victory against the giant squid, figure out which board will win first. What will your final score be if you choose that board?

In [39]:
test_input = get_puzzle_input('test_input_4.txt', remove_blank_lines=False)
real_input = get_puzzle_input('input_4.txt', remove_blank_lines=False)

In [40]:
class Board:
    def __init__(self, grid):
        self.grid_size = len(grid)
        self.parse_grid(grid)
        self.reset()

    def parse_grid(self, grid):
        self.numbers = {}
        self.locations = {}
        for row_idx, line in enumerate(grid):
            row = re.split('\s+', line)
            for col_idx, num in enumerate(row):
                location = (col_idx, row_idx)
                num = int(num)
                self.numbers[num] = location
                self.locations[location] = num
    
    def reset(self):
        self.dabbed = Counter()
        self.unmarked = set(self.numbers.keys())
        
    def dab(self, number):
        if number not in self.unmarked:
            return None
        
        self.unmarked.remove(number)

        location = self.numbers[number]
        
        col_idx = ('COL', location[0])
        row_idx = ('ROW', location[1])
        self.dabbed[col_idx] += 1
        self.dabbed[row_idx] += 1
        
        col_win = self.dabbed[col_idx] == self.grid_size
        row_win = self.dabbed[row_idx] == self.grid_size

        if not col_win and not row_win:
            return None

        winning_line = []
        
        for line_idx in range(self.grid_size):
            num_location = (location[0], line_idx) if col_win else (line_idx, location[1])
            winning_line.append(self.locations[num_location])
        
        return winning_line     

In [41]:
# Test winning row
input = [
    '14 21 17 24  4',
    '10 16 15  9 19',
    '18  8 23 26 20',
    '22 11 13  6  5',
    '2  0 12  3  7'
]
board = Board(input)
for num in [14, 21, 17, 24]:
    board.dab(num)
assert board.dab(4) == [14, 21, 17, 24, 4]

In [42]:
# Test winning col
board.reset()
for num in [17, 15, 13, 12]:
    board.dab(num)
assert board.dab(23) == [17, 15, 23, 13, 12]

In [43]:
class Game:
    def __init__(self, lines):
        self.parse_input(lines)

    def parse_input(self, lines):
        self.numbers = [int(i) for i in lines[0].split(',')]
        self.boards = []
        grids = deque(lines[1:])
        while len(grids) >= 6:
            grids.popleft()
            grid = [grids.popleft() for _ in range(5)]
            self.boards.append(Board(grid))
        
        self.in_play_boards = set(range(len(self.boards)))
    
    def play(self):
        for num in self.numbers:
            for board in self.boards:
                win = board.dab(num)
                if win:
                    return num, win, sum(board.unmarked)

    def play_part_2(self):
        for num in self.numbers:
            for board_idx in sorted(list(self.in_play_boards)):
                board = self.boards[board_idx]
                win = board.dab(num)
                if win:
                    # print(f'{num} {board_idx} {win} {sum(board.unmarked)}')
                    self.in_play_boards.remove(board_idx)
                    if not self.in_play_boards:
                        return num, win, sum(board.unmarked)

In [46]:
test_game = Game(test_input)
test_game.play()

(24, [14, 21, 17, 24, 4], 188)

In [47]:
real_game = Game(real_input)
real_game.play()

(35, [75, 95, 35, 6, 47], 726)

In [48]:
35 * 726

25410

### Day 4, part 2

Figure out which board will win last. Once it wins, what would its final score be?

In [49]:
test_game = Game(test_input)
test_game.play_part_2()

(13, [0, 13, 7, 10, 16], 148)

In [50]:
real_game = Game(real_input)
real_game.play_part_2()

(14, [95, 18, 52, 14, 22], 195)

In [51]:
14 * 195

2730

### Day 4 Observations

Make sure test and production run the same code. I originally was loading the test input from the command line and the real input from a file. I had forgotten that this was stripping blank lines by default hence the Bingo boards were being mixed up. This was doubly confusing given that the first part gave the correct answer by chance and the bug was only found in running the second part which assumed we were developing on solid ground.

## Day 5 Hydrothermal Venture

https://adventofcode.com/2021/day/5

To avoid the most dangerous areas, you need to determine the number of points where at least two lines overlap. In the above example, this is anywhere in the diagram with a 2 or larger - a total of 5 points.

Consider only horizontal and vertical lines. At how many points do at least two lines overlap?

In [52]:
test_input = get_puzzle_input('test_input_5.txt')
real_input = get_puzzle_input('input_5.txt')

In [55]:
Point = namedtuple('Point', ('x', 'y'))

In [56]:
# Part 1
def intersected_points(point_a, point_b):
    lower, upper = sorted([point_a, point_b])
    if upper.x > lower.x:
        return [Point(x, upper.y) for x in range(lower.x, upper.x + 1)]
    
    return [Point(upper.x, y) for y in range(lower.y, upper.y + 1)]

In [57]:
# Part 2 - handles diagonal lines
def intersected_points(point_a, point_b):
    line_len = 1
    if point_a.x > point_b.x:
        x_incr = -1
        line_len = point_a.x - point_b.x
    elif point_a.x < point_b.x:
        x_incr = 1
        line_len = point_b.x - point_a.x
    else:
        x_incr = 0

    if point_a.y > point_b.y:
        y_incr = -1
        line_len = point_a.y - point_b.y
    elif point_a.y < point_b.y:
        y_incr = 1
        line_len = point_b.y - point_a.y
    else:
        y_incr = 0

    return {
        Point(point_a.x + x_incr * val, point_a.y + y_incr * val)
        for val
        in range(line_len + 1)
    } 

In [58]:
assert intersected_points(Point(1, 2), Point(1, 4)) == {Point(x=1, y=2), Point(x=1, y=3), Point(x=1, y=4)}
assert intersected_points(Point(1, 4), Point(1, 2)) == {Point(x=1, y=2), Point(x=1, y=3), Point(x=1, y=4)}
assert intersected_points(Point(1, 4), Point(1, 4)) == {Point(x=1, y=4)}
assert intersected_points(Point(10, 4), Point(12, 4)) == {Point(x=10, y=4), Point(x=11, y=4), Point(x=12, y=4)}
assert intersected_points(Point(1, 2), Point(4, 5)) == {Point(x=1, y=2), Point(x=2, y=3), Point(x=3, y=4), Point(x=4, y=5)}

In [59]:
class Game:
    regexp = re.compile('^([-0-9]+),([-0-9]+) -> ([-0-9]+),([-0-9]+)$')

    def __init__(self, input):
        self.parse_input(input)
    
    def parse_input(self, puzzle_input):
        self.lines = []
        for line in puzzle_input:
            match = self.regexp.match(line)
            if match:
                groups = match.groups()
                point_a = Point(int(groups[0]), int(groups[1]))
                point_b = Point(int(groups[2]), int(groups[3]))
                self.lines.append((point_a, point_b))

    def play(self, ignore_diagonals=True):
        points = Counter()
        for (point_a, point_b) in self.lines:
            if ignore_diagonals:
                # Part 1
                if point_a.x != point_b.x and point_a.y != point_b.y:
                    continue
            for point in intersected_points(point_a, point_b):
                points[point] += 1
        
        danger_points = [
            point
            for (point, val)
            in points.items()
            if val > 1
        ]
        
        return len(danger_points)

In [60]:
test_game = Game(test_input)
test_game.play()

5

In [61]:
real_game = Game(real_input)
real_game.play()

5092

### Day 5 Part 2

You still need to determine the number of points where at least two lines overlap. In the above example, this is still anywhere in the diagram with a 2 or larger - now a total of 12 points.

Consider all of the lines. At how many points do at least two lines overlap?

In [62]:
test_game = Game(test_input)
test_game.play(ignore_diagonals=False)

12

In [63]:
real_game = Game(real_input)
real_game.play(ignore_diagonals=False)

20484

### Day 5 Observations

Intersecting points implementation is a bit clunky but - hey ho.

## Day 6 Lantern Fish

https://adventofcode.com/2021/day/6

In this example, after 18 days, there are a total of 26 fish. After 80 days, there would be a total of 5934.

Find a way to simulate lanternfish. How many lanternfish would there be after 80 days?

In [67]:
class LanternFish:
    CYCLE_LENGTH = 6
    DEFAULT_START = 8

    def __init__(self, start_of_cycle=DEFAULT_START):
        self.start_of_cycle = start_of_cycle
        self.current = start_of_cycle

    def new_day(self):
        self.current -= 1
        if self.current == -1:
            self.current = self.CYCLE_LENGTH
            child = LanternFish()
            return child
        else:
            return None

In [68]:
class SingleFishGame:
    """Only need to solve for each unique initial state as any similar fish
    will result in the same number of descendents.
    """
    def __init__(self, start_of_cycle):
        self.fishes = [LanternFish(start_of_cycle=start_of_cycle)]
    
    def play(self, num_days):
        for _ in range(num_days):
            new_fish = []
            for fish in self.fishes:
                child = fish.new_day()
                if child:
                    new_fish.append(child)
            
            self.fishes.extend(new_fish)
        
        return len(self.fishes) 

In [69]:
%%time

num_descendents_per_state = {}

# Loop over all the possible initial states
for i in range(7):
    game = SingleFishGame(i)
    total = game.play(80)
    num_descendents_per_state[i] = total

num_descendents_per_state

CPU times: user 35.8 ms, sys: 1.69 ms, total: 37.5 ms
Wall time: 36 ms


{0: 1421, 1: 1401, 2: 1191, 3: 1154, 4: 1034, 5: 950, 6: 905}

In [70]:
initial_count = Counter()
for i in [5,3,2,2,1,1,4,1,5,5,1,3,1,5,1,2,1,4,1,2,1,2,1,4,2,4,1,5,1,3,5,4,3,3,1,4,1,3,4,4,1,5,4,3,3,2,5,1,1,3,1,4,3,2,2,3,1,3,1,3,1,5,3,5,1,3,1,4,2,1,4,1,5,5,5,2,4,2,1,4,1,3,5,5,1,4,1,1,4,2,2,1,3,1,1,1,1,3,4,1,4,1,1,1,4,4,4,1,3,1,3,4,1,4,1,2,2,2,5,4,1,3,1,2,1,4,1,4,5,2,4,5,4,1,2,1,4,2,2,2,1,3,5,2,5,1,1,4,5,4,3,2,4,1,5,2,2,5,1,4,1,5,1,3,5,1,2,1,1,1,5,4,4,5,1,1,1,4,1,3,3,5,5,1,5,2,1,1,3,1,1,3,2,3,4,4,1,5,5,3,2,1,1,1,4,3,1,3,3,1,1,2,2,1,2,2,2,1,1,5,1,2,2,5,2,4,1,1,2,4,1,2,3,4,1,2,1,2,4,2,1,1,5,3,1,4,4,4,1,5,2,3,4,4,1,5,1,2,2,4,1,1,2,1,1,1,1,5,1,3,3,1,1,1,1,4,1,2,2,5,1,2,1,3,4,1,3,4,3,3,1,1,5,5,5,2,4,3,1,4]:
    initial_count[i] += 1
    
initial_count

Counter({5: 42, 3: 43, 2: 52, 1: 109, 4: 54})

In [71]:
sum(count * num_descendents_per_state[state] for state, count in initial_count.items())

359999

### Day 6 Part 2

And on day 256 how many will there be?

In [72]:
class LanternFish:
    """Any fish born on the same day will have the same number of descendents
    so only need to track one."""
    CYCLE_LENGTH = 6
    DEFAULT_START = 8

    def __init__(self, start_of_cycle=DEFAULT_START):
        self.start_of_cycle = start_of_cycle
        self.current = start_of_cycle
        self.untracked_parents = []
        self.num_descendents = 0

    def gives_birth(self):
        self.current -= 1
        if self.current == -1:
            self.current = self.CYCLE_LENGTH
            return True
        else:
            return False
    
    def add_parent(self, parent):
        self.untracked_parents.append(parent)
        parent.add_descendent()

    def add_descendent(self):
        self.num_descendents += 1
        for parent in self.untracked_parents:
            parent.add_descendent()

In [73]:
class SingleFishGame:
    def __init__(self, start_of_cycle):
        self.fishes = [LanternFish(start_of_cycle=start_of_cycle)]
        self.day = 0
    
    def play(self, num_days):
        for day_num in range(num_days):
            new_fish = []
            child = None
            for fish in self.fishes:
                gives_birth = fish.gives_birth()
                if not gives_birth:
                    continue
                
                if child:
                    child.add_parent(fish)
                else:
                    child = LanternFish()
                    new_fish.append(child)
            
            self.fishes.extend(new_fish)
        
        return len(self.fishes) + sum(fish.num_descendents for fish in game.fishes)

In [75]:
%%time

part_2_num_descendents_per_state = {}
for i in range(1, 6):
    print(f'Calculating for initial state {i}...')
    game = SingleFishGame(i)
    total = game.play(256)
    part_2_num_descendents_per_state[i] = total

part_2_num_descendents_per_state

Calculating for initial state 1...
Calculating for initial state 2...
Calculating for initial state 3...
Calculating for initial state 4...
Calculating for initial state 5...
CPU times: user 1h 6min 9s, sys: 3.29 s, total: 1h 6min 13s
Wall time: 1h 6min 16s


{1: 6206821033, 2: 5617089148, 3: 5217223242, 4: 4726100874, 5: 4368232009}

In [77]:
sum(count * part_2_num_descendents_per_state[state] for state, count in initial_count.items())

1631647919273

### Day 6 Observations

Returns in one hour - wow. As I was running it for the first index I could see that it was going to return (from printing out the day being calculated). I don't think there was a need to rerun it for all the initial states - Could have, for instance derived state 2 from state 1 by eliminating all state 8s (newborns) and likewise for state 3 - not counting 7s and 8s. Perhaps I could say that state 2 on day 265 is state 1 on day 264 etc so only need one run through. Could have also looked for repeated patterns - but - there comes the point when you're bunged up with a cold and you're happy to have breakfast and wait.

### Day 6 Revisited

A few days later I had a look at this again and realised a far easier approach of mapping states to future states a week down the line. So in general given a fish in state n, in a weeks time it will result in a fish in state n (itself) and a fish in state n+2 (its child) unless the initial state is 7 or 8 (a newborn)

In [2]:
def num_fish_from_state(initial_state, num_days):
    seven_day_state_map = {
        0: (0, 2),
        1: (1, 3),
        2: (2, 4),
        3: (3, 5),
        4: (4, 6),
        5: (5, 7),
        6: (6, 8),
        7: (0, ),
        8: (1, ),
    }
    counts = [0] * 9
    
    num_weeks = num_days // 7
    remaining_days = num_days - num_weeks * 7
    
    initial_state -= remaining_days
    counts[initial_state % 7] = 1
    if initial_state < 0:
        # The newly spawned fish
        counts[8 + initial_state + 1] = 1
    
    # Go up in weeks
    for i in range(num_weeks):
        for state, num in enumerate(list(counts)):
            new_states = seven_day_state_map[state]
            counts[state] -= num
            for new_state in new_states:
                counts[new_state] += num
    
    return sum(counts)
  

In [3]:
%%time

part_2_lookup_improved = {}

for i in range(1, 6):
    part_2_lookup_improved[i] = num_fish_from_state(i, 256)

part_2_lookup_improved

CPU times: user 1.7 ms, sys: 7 µs, total: 1.71 ms
Wall time: 1.71 ms


{1: 6206821033, 2: 5617089148, 3: 5217223242, 4: 4726100874, 5: 4368232009}

From 1 hour to 1 millisecond! It's interesting that the puzzle setter went for 256 days. If I think about the question without thinking of the implementation I might have been tempted on a year (365 days). If it were a year the solution I went for on the day wouldn't have returned. On the otherhand if it were any less I wouldn't have been suitably punished for a slow implementation. If it were 200 days for example - my solution would have returned in under 30 seconds and returned in 6 minutes for 230 days - both being far quicker to wait than to rewrite. 256 seems a sweet spot that allows less lazy people the oportunity to rewrite and come up with a more elegant and satisfying solution whilst keeping us lazy folk in the game.

## Day 7 The Treachery of Whales

https://adventofcode.com/2021/day/7

Determine the horizontal position that the crabs can align to using the least fuel possible. How much fuel must they spend to align to that position?

In [78]:
test_positions = [16,1,2,0,4,2,7,1,2,14]
real_positions = [1101,1,29,67,1102,0,1,65,1008,65,35,66,1005,66,28,1,67,65,20,4,0,1001,65,1,65,1106,0,8,99,35,67,101,99,105,32,110,39,101,115,116,32,112,97,115,32,117,110,101,32,105,110,116,99,111,100,101,32,112,114,111,103,114,97,109,10,485,546,350,100,791,199,115,144,649,41,1656,163,903,71,384,30,2,251,554,210,434,206,546,759,258,54,1478,48,438,601,326,5,1017,165,168,201,622,864,1338,24,1074,545,499,484,264,345,332,869,297,711,674,346,1139,317,875,242,725,250,1619,1408,956,380,366,187,1034,1555,467,170,114,1136,150,183,304,44,37,333,791,34,540,716,1923,342,6,922,18,24,1189,59,1726,636,442,426,1089,526,298,386,296,623,80,272,240,406,628,238,409,302,35,404,92,48,157,1545,409,1382,151,1656,3,76,14,115,566,650,197,448,573,161,86,140,875,128,319,4,822,530,189,247,667,82,316,274,110,206,1012,166,639,579,459,284,200,16,24,147,743,113,1562,387,60,84,797,14,30,1015,508,88,113,685,658,257,1507,348,30,808,416,9,835,671,16,474,885,230,47,463,1324,1263,183,603,739,0,296,789,1411,339,27,1154,31,882,409,646,92,153,147,974,497,308,85,311,135,627,811,295,698,2,20,1170,789,702,1194,1390,432,257,715,958,150,1295,144,1193,607,67,929,383,1051,1231,393,190,380,1203,1090,1238,143,206,210,1004,304,1305,392,143,1379,665,806,452,185,4,1,201,1104,633,274,493,472,141,674,1261,106,587,244,903,91,158,69,137,922,778,143,692,160,474,7,304,824,657,15,1110,806,295,1565,1162,358,725,877,440,690,13,69,111,304,300,493,249,105,746,20,163,561,913,558,252,13,193,508,12,845,120,205,154,1582,349,1471,529,268,23,689,6,776,565,401,0,623,186,62,95,148,275,1,137,320,0,19,1803,10,100,652,750,226,484,180,46,310,446,667,543,277,139,265,74,171,87,1753,337,162,59,1339,1040,1287,1084,192,169,50,1557,81,1120,271,167,977,76,295,12,54,710,36,364,521,989,1634,720,1031,1204,355,380,859,633,223,1207,221,31,138,1305,779,1026,52,92,216,221,0,980,130,1197,585,1213,63,157,213,993,1123,588,450,256,1021,90,1420,47,386,843,1188,1466,807,596,416,23,32,62,1289,317,368,491,907,1386,114,1620,39,344,1342,43,281,12,1202,257,1357,203,465,174,350,833,125,54,390,687,339,628,819,261,1341,840,643,414,82,373,428,1315,570,1070,686,893,70,728,70,358,1233,189,1247,244,1043,1135,42,531,962,35,30,1462,946,856,145,386,1134,1071,379,740,175,1205,234,354,5,1028,506,58,433,1055,749,854,99,298,1248,619,62,181,258,42,130,1698,1313,672,129,222,127,636,846,24,1324,946,622,689,168,329,301,458,173,591,772,93,282,8,320,106,233,412,556,2,522,369,8,1371,899,503,568,667,1199,92,115,899,952,81,629,175,274,763,204,339,236,317,257,731,1082,1724,211,516,165,91,334,1216,101,21,1340,235,336,1351,723,1745,183,841,104,172,1080,180,493,798,1468,45,1627,59,58,368,560,166,1125,136,26,1238,1580,420,1732,155,55,293,751,194,1723,175,11,30,10,307,57,66,704,285,685,241,565,368,50,181,1047,147,420,1341,20,37,400,798,476,1060,642,134,140,502,254,997,910,636,179,22,612,55,237,258,48,205,412,155,910,192,262,9,91,766,1426,71,5,315,285,186,629,422,1289,397,52,860,1390,106,887,1285,1196,684,36,703,199,4,277,151,82,293,1047,455,21,935,630,736,118,13,30,584,453,1446,381,585,810,177,1028,280,281,184,78,673,126,410,872,524,78,188,121,394,201,1764,609,350,706,428,88,783,189,643,305,516,259,582,309,985,338,21,235,73,44,585,71,983,175,1336,1056,10,8,537,701,1653,657,70,1242,442,52,973,203,173,959,964,272,348,3,567,714,1466,382,129,613,1042,686,461,57,523,740,726,149,1490,867,44,379,1270,547,649,1103,912,1354,985,458,887,603,1016,317,499,690,829,1231,364,772,29,57,357,467,484,202,150,109,95,414,444,383,62,124,645,723,772,881,1553,413,123,248,1085,453,260,214,113,1874,482,942,235,899,122,171,127,913,424,406,49,97,1848,295,1152,111,350,54,1160,2,16,156,448,394,740,49,1237,548,206,1206,775,748,728,48,238,148,109,18,56,64,515,163,609,273,301,396,207,51,478,1183,864,772,450,222,1387,269,40,87,426,164,1270,21,347,316,331,408,914,1046,173,48,398,177,431,47,1055,221,513,226,84,285,566,270,333,343,480,1802,101,683,168,1347,582,80,22,329,350,108,379,14,53,349,43,435,195,102,168,338]

In [79]:
len(real_positions), max(real_positions), min(real_positions)

(1000, 1923, 0)

OK, only roughly 2 million calculations to brute force every position. Let's start there

In [80]:
def crab_fuel_used_part_1(curr_pos, new_pos):
    return abs(curr_pos - new_pos)

def crab_fuel_used_part_2(curr_pos, new_pos):
    diff = abs(curr_pos - new_pos)
    return diff * (diff + 1) // 2

def best_alignment(positions, crab_fuel_used):
    best_pos = None
    best_fuel_used = None
    max_pos = max(positions)
    min_pos = min(positions)
    
    for x in range (min_pos, max_pos + 1):
        fuel_used = sum(crab_fuel_used(pos, x) for pos in positions)
        if best_fuel_used is None or fuel_used < best_fuel_used:
            best_pos = x
            best_fuel_used = fuel_used
    
    return best_pos, best_fuel_used

In [81]:
best_alignment(test_positions, crab_fuel_used_part_1), best_alignment(real_positions, crab_fuel_used_part_1)

((2, 37), (339, 343468))

### Day 7 Part 2

And now the crabs use an extra unit of fuel every time they move

In [82]:
best_alignment(test_positions, crab_fuel_used_part_2), best_alignment(real_positions, crab_fuel_used_part_2)

((5, 168), (478, 96086265))

### Day 7 Observations

Suspiciously straight forward - the calm before the storm? We are heading towards a maze! I thought the strategy pattern worked quite well here.

## Day 8 Seven Segment Search

https://adventofcode.com/2021/day/8

Because the digits 1, 4, 7, and 8 each use a unique number of segments, you should be able to tell which combinations of signals correspond to those digits. Counting only digits in the output values (the part after | on each line), in the above example, there are 26 instances of digits that use a unique number of segments (highlighted above).

In the output values, how many times do digits 1, 4, 7, or 8 appear?

In [1]:
def get_output_digits(puzzle_line):
    output = puzzle_line.split(' ')[11:]
    return [frozenset(s) for s in output]

def get_input_signals(puzzle_line):
    input = puzzle_line.split(' ')[:10]
    return [frozenset(s) for s in input]  

In [4]:
test_input = get_puzzle_input('test_input_8.txt')
real_input = get_puzzle_input('input_8.txt')

In [5]:
def solve_part_1(puzzle_input):
    unique_num_count = 0
    for line in puzzle_input:
        output_digits = get_output_digits(line)
        for digit in output_digits:
            if len(digit) in (2, 3, 4, 7):
                unique_num_count += 1
    
    return unique_num_count

In [6]:
solve_part_1(test_input), solve_part_1(real_input)

(26, 264)

### Day 8, part 2

```
Considering the line
acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab | cdfeb fcadb cdfeb cdbaf

After some careful analysis, the mapping between signal wires and segments only make sense in the following configuration:

 dddd
e    a
e    a
 ffff
g    b
g    b
 cccc

So, the unique signal patterns would correspond to the following digits:

acedgfb: 8
cdfbe: 5
gcdfa: 2
fbcad: 3
dab: 7
cefabd: 9
cdfgeb: 6
eafb: 4
cagedb: 0
ab: 1
```

For each entry, determine all of the wire/segment connections and decode the four-digit output values. What do you get if you add up all of the output values?

In [8]:
def get_digit_map(signal_pattern):
    one = [s for s in signal_pattern if len(s) == 2][0]
    four = [s for s in signal_pattern if len(s) == 4][0]
    seven = [s for s in signal_pattern if len(s) == 3][0]
    eight = [s for s in signal_pattern if len(s) == 7][0]
    
    # 0, 6 and 9
    six_segments = [s for s in signal_pattern if len(s) == 6]
    
    six = [s for s in six_segments if s & seven != seven][0]
    six_segments.remove(six)
    nine = [s for s in six_segments if s & four == four][0]
    six_segments.remove(nine)
    zero = six_segments[0]

    # 3, 2 and 5
    five_segments = [s for s in signal_pattern if len(s) == 5]
    five = [s for s in five_segments if len(six - s)  == 1][0]
    five_segments.remove(five)
    
    three = [s for s in five_segments if s & one == one][0]
    five_segments.remove(three)
    two = five_segments[0]
    
    return {
        zero: 0,
        one: 1,
        two: 2,
        three: 3,
        four: 4,
        five: 5,
        six: 6,
        seven: 7,
        eight: 8,
        nine: 9
    }

In [9]:
# From the above example:
test_line = 'acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab | cdfeb fcadb cdfeb cdbaf'
input_signals = get_input_signals(test_line)
digit_map = get_digit_map(input_signals)

expected = {
    frozenset('acedgfb'): 8,
    frozenset('cdfbe'): 5,
    frozenset('gcdfa'): 2,
    frozenset('fbcad'): 3,
    frozenset('dab'): 7,
    frozenset('cefabd'): 9,
    frozenset('cdfgeb'): 6,
    frozenset('eafb'): 4,
    frozenset('cagedb'): 0,
    frozenset('ab'): 1,
}

assert sorted(digit_map.items(), key=lambda x: x[1]) == sorted(expected.items(), key=lambda x: x[1])

In [10]:
def solve_part_2(puzzle_input):
    total = 0
    for line in puzzle_input:
        input_signals = get_input_signals(line)
        output_digits = get_output_digits(line)
        
        digit_map = get_digit_map(input_signals)
        
        decoded_digits = [digit_map[d] for d in output_digits]
        decoded_figure = int(''.join(map(str, decoded_digits)))
        total += decoded_figure
    
    return total

In [11]:
solve_part_2(test_input), solve_part_2(real_input)

(61229, 1063760)

### Day 8 Observations

We have a tech talk session on Wednesdays - so I thought for the Christmas special that we'd do today's AOC puzzle as a group swarm programming session. The session's only half an hour so I wanted to check what was achievable to explain and complete in that time before hand (that is to say I cheated and completed it prior to the session.) To fit in the time and to choose something interesting I concentrated on part 2 with only the `get_digit_map` function (the hard bit) not implemented. So, there was a lot of boilerplate already there to test and execute the function. Here's some observations from the session:

- Some people just really don't like puzzles. Problem solving as a group can ellicit a range of negative emotions - from a more neutral - just a waste of time, to feeling the pressure of performing in front of peers. It's important to give people an out by either saying what the session is going to be about up front (I didn't) or telling people that it's fine to drop off if they want to get the time back (I did.)

- The read though and part one of these puzzles are both really important. You think through possible implementations as you're reading it; you're potentially second guessing what's going to be in the second part. Completing the first part gives you a good feel for what you're working with, and you take all this into the second part. Tackling one aspect of the second part involved me paraphrasing the problem up to that point to the group and together with the boilerplate code they had to jump right in at the deep end without a chance to really think it through. This gap in prior knowledge also either made it look like I was exceptionally clever (unlikely) or that I was pretending to be exceptionally clever (very likely). If I were doing it again I think I would do it blind on part 1 of a problem; giving everyone five minutes to read it through first. I'd obviously need to put to one side my own fear of failing in front of peers - failing to solve it can be just as interesting!

- Even though they were thrown in at the deep end - they did it! (their solution is above). I really tried not to prompt - honest - just to type.

## Day 9 Smoke Basin

https://adventofcode.com/2021/day/9

The risk level of a low point is 1 plus its height. In the above example, the risk levels of the low points are 2, 1, 6, and 6. The sum of the risk levels of all low points in the heightmap is therefore 15.

Find all of the low points on your heightmap. What is the sum of the risk levels of all low points on your heightmap?

In [8]:
test_input = get_puzzle_input('test_input_9.txt')
real_input = get_puzzle_input('input_9.txt')

In [9]:
@dataclass(frozen=True)
class Point:
    x: int
    y: int
    
    @property
    def adjacent_points(self):
        return [
            Point(self.x - 1, self.y),
            Point(self.x + 1, self.y),
            Point(self.x, self.y + 1),
            Point(self.x, self.y - 1),
        ]


In [10]:
class Grid:
    def __init__(self, puzzle_input):
        self.points = {}
        self.max_x = len(puzzle_input[0]) - 1
        self.max_y = len(puzzle_input) - 1

        for y, line in enumerate(puzzle_input):
            for x, val in enumerate(line):
                self.points[Point(x, y)] = int(val)
    
    def get_adjacent_points(self, point):
            adjacent_points = point.adjacent_points
            adjacent_points = [
                point
                for point in adjacent_points
                if point.x >= 0 and point.x <= self.max_x
            ]
            adjacent_points = [
                point
                for point in adjacent_points
                if point.y >= 0 and point.y <= self.max_y
            ]
            return adjacent_points
        

    def play_part_1(self):
        risk = 0
        for point, val in self.points.items():
            adjacent_values = [
                self.points[point]
                for point
                in self.get_adjacent_points(point)
            ]
            
            if all([val < adjacent_val for adjacent_val in adjacent_values]):
                risk += 1 + val
        
        return risk

    def find_basin(self, remaining_points):
        # pick a starting point
        basin = {list(remaining_points)[0]}
        
        while True:
            added_points = set()
            for point in basin:
                adjacent_points = self.get_adjacent_points(point)
                adjacent_points = [point for point in adjacent_points if point in remaining_points]
                adjacent_points = {point for point in adjacent_points if self.points[point] < 9}
                added_points = added_points | adjacent_points

            new_basin = basin | added_points
            if new_basin == basin:
                return new_basin

            basin = new_basin
    
    def find_basins(self):
        basins = []
        remaining_points = {point for point, val in self.points.items() if val < 9}
        
        while remaining_points:
            basin = self.find_basin(remaining_points)
            basins.append(basin)
            remaining_points -= basin
        
        return basins 
    
    def play_part_2(self):
        basins = self.find_basins()
        sizes = [len(basin) for basin in basins]
        sizes = list(reversed(sorted(sizes)))
        return sizes[0] * sizes[1] * sizes[2]

In [11]:
test_grid = Grid(test_input)
test_grid.play_part_1()

15

In [12]:
real_grid = Grid(real_input)
real_grid.play_part_1()

478

### Day 9 part 2

Find the three largest basins and multiply their sizes together. In the above example, this is 9 * 14 * 9 = 1134.

What do you get if you multiply together the sizes of the three largest basins?

In [13]:
test_grid = Grid(test_input)
test_grid.play_part_2()

1134

In [14]:
real_grid = Grid(real_input)
real_grid.play_part_2()

1327014

### Day 9 Observations

I find it difficult to spell the word, 'adjacent'.

## Day 10 Syntax Scoring

https://adventofcode.com/2021/day/10

In the above example, an illegal ) was found twice (2*3 = 6 points), an illegal ] was found once (57 points), an illegal } was found once (1197 points), and an illegal > was found once (25137 points). So, the total syntax error score for this file is 6+57+1197+25137 = 26397 points!

Find the first illegal character in each corrupted line of the navigation subsystem. What is the total syntax error score for those errors?

In [4]:
test_input = get_puzzle_input('test_input_10.txt')
real_input = get_puzzle_input('input_10.txt')

In [5]:
def find_syntax_error(line):
    stack = []
    pair_map = {
        '{': '}',
        '(': ')',
        '[': ']',
        '<': '>', 
    }
    close_map = {v: n for (n, v) in pair_map.items()}
    openings = set(pair_map.keys())
    endings = set(pair_map.values())
    
    for char in line:
        if char in openings:
            stack.append(char)
        
        if char in endings:
            if not stack:
                return (None, char)
            opening = stack.pop()
            expected = pair_map[opening]
            if char != expected:
                return (expected, char)
        
    return None

In [6]:
assert find_syntax_error('{}') is None
assert find_syntax_error('{]') == ('}', ']')

In [7]:
def play_part_1(input):
    total = 0
    scores = {
        ')': 3,
        ']': 57,
        '}': 1197,
        '>': 25137
    }
    for line in input:
        error = find_syntax_error(line)
        if error:
            error_char = error[1]
            score = scores[error_char]
            total += score
    
    return total

In [8]:
play_part_1(test_input), play_part_1(real_input)

(26397, 288291)

### Day 10, part 2

Autocomplete tools are an odd bunch: the winner is found by sorting all of the scores and then taking the middle score. (There will always be an odd number of scores to consider.) In this example, the middle score is 288957 because there are the same number of scores smaller and larger than it.

Find the completion string for each incomplete line, score the completion strings, and sort the scores. What is the middle score?

In [9]:
def find_completion_sequence(line):
    stack = []
    pair_map = {
        '{': '}',
        '(': ')',
        '[': ']',
        '<': '>', 
    }
    close_map = {v: n for (n, v) in pair_map.items()}
    openings = set(pair_map.keys())
    endings = set(pair_map.values())
    
    for char in line:
        if char in openings:
            stack.append(char)
        
        if char in endings:
            if not stack:
                return []
            opening = stack.pop()
            expected = pair_map[opening]
            if char != expected:
                return []
        
    return [pair_map[char] for char in reversed(stack)]

In [10]:
def play_part_2(input):
    line_scores = []
    scores = {
        ')': 1,
        ']': 2,
        '}': 3,
        '>': 4
    }
    for line in input:
        line_score = 0
        completion_sequence = find_completion_sequence(line)
        for char in completion_sequence:
            line_score *= 5
            line_score += scores[char]
        
        if line_score:
            line_scores.append(line_score)
    
    line_scores = sorted(line_scores)
    median = line_scores[len(line_scores) // 2]
    return median

In [11]:
play_part_2(test_input), play_part_2(real_input)

(288957, 820045242)

### Day 10 Observations

It's frustrating when you know how to do part two but you've really got to go and catch a train. It would have been more elegant to represent the syntax errors as exceptions and so using the same function for parts 1 and 2.